In [223]:
import camelot
import pandas as pd
import numpy as np

In [224]:
# https://stackoverflow.com/questions/45846765/efficient-way-to-unnest-explode-multiple-list-columns-in-a-pandas-dataframe

def explode(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

In [244]:
tables = camelot.read_pdf('Sparkassenrangliste_2018.pdf',pages='all')

# Creating new df to append to
df = pd.DataFrame()

# Iterate over each table in the 10 pages
for table in tables:
    table = table.df
    table.columns = ['Rang','Institut','Sitz','Verband','Bilanzsumme gemäß Bilanzstatistik','Kundeneinlagen','Spareinlagen', 'Kundenkredite','Anzahl Mitarbeiter','Sparkassenstellen (einschließlich SB)']
    table.drop(index=[0,1,2],inplace=True)
    table = table.apply(lambda text: text.str.split('\n'))
    table.reset_index(inplace=True)
    table.drop(columns='index',inplace=True)
    table = explode(table,list(table.columns))
    df = df.append(table)

df.set_index('Rang',inplace=True)

# Get rid of '.' to convert to numeric
df[list(df.columns)[3:]] = df[list(df.columns)[3:]].apply(lambda number: number.str.replace('.',''))

# Convert colukns into integers
for column in list(df.columns)[3:]:
    df[column] = pd.to_numeric(df[column])

# Remove any potential whitespace to have clean text
for column in list(df.columns)[:3]:
    df[column] = df[column].str.strip()
    
# Save df as csv for later use in the scarper
df.to_csv('sparkassen_rangliste_cleaned.csv')

In [249]:
sparkassen = df['Institut'].to_list()
google_search_url = 'https://www.google.de/search?q='

In [253]:
for sparkasse in sparkassen:
    print(google_search_url+sparkasse.replace(' ','+'))

https://www.google.de/search?q=Hamburger+Sparkasse
https://www.google.de/search?q=Sparkasse+KölnBonn
https://www.google.de/search?q=Kreissparkasse+Köln
https://www.google.de/search?q=Frankfurter+Sparkasse
https://www.google.de/search?q=Stadtsparkasse+München
https://www.google.de/search?q=Sparkasse+Hannover
https://www.google.de/search?q=Mittelbrandenburgische+Sparkasse+in+Potsdam
https://www.google.de/search?q=Sparkasse+Pforzheim+Calw
https://www.google.de/search?q=Ostsächsische+Sparkasse+Dresden
https://www.google.de/search?q=Nassauische+Sparkasse
https://www.google.de/search?q=Kreissparkasse+München+Starnberg+Ebersberg
https://www.google.de/search?q=Stadtsparkasse+Düsseldorf
https://www.google.de/search?q=Die+Sparkasse+Bremen+AG
https://www.google.de/search?q=Sparkasse+Aachen
https://www.google.de/search?q=Sparkasse+Nürnberg
https://www.google.de/search?q=Kreissparkasse+Ludwigsburg
https://www.google.de/search?q=Landessparkasse+zu+Oldenburg
https://www.google.de/search?q=Sparkasse+M